In [ ]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-qyhemr47
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-qyhemr47
  Resolved https://github.com/huggingface/transformers to commit 15bd3e61f8d3680ca472c9314ad07584d20f7b81
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10450848 sha256=4360c653ab2663d9f02e8649eb26f0a1eadca29fa7d5cd321f0a1fc5e16b45c2
  Stored in directory: /tmp/pip-ephem-wheel-cache-uj_3ixai/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade torch accelerate

In [ ]:
import pandas as pd
import json
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import argparse

# Set the seed for reproducibility
RANDOM_SEED = 42

# Load the dataset
df = pd.read_csv("/content/drive/MyDrive/classification_results_classified_20250105_133618.csv")
df

,statement,true_label,predicted_label,is_correct
0,Driving Anxiety I tend to get very anxious whi...,anxiety,depression,False
1,Department at Work Is Passing Around Stomach B...,anxiety,anxiety,True
2,Exam on Saturday. Saturday's Starting Presenta...,anxiety,anxiety,True
3,Its 2 am rn and i’m freaking out that i’m forg...,anxiety,anxiety,True
4,Son has weird bump on shoulder I'm waiting for...,anxiety,depression,False
...,...,...,...,...
2556,I have tried every single day this week I have...,suicidal,depression,False
2557,I am sorry to post this to other people dealin...,suicidal,normal,False
2558,"someone help me, my dad died yesterday, my mom...",suicidal,depression,False
2559,I am ready to go. I do not want to be here any...,suicidal,anxiety,False


In [ ]:
df = df.rename(columns={'true_label': 'status'})
df

,statement,status,predicted_label,is_correct
0,Driving Anxiety I tend to get very anxious whi...,anxiety,depression,False
1,Department at Work Is Passing Around Stomach B...,anxiety,anxiety,True
2,Exam on Saturday. Saturday's Starting Presenta...,anxiety,anxiety,True
3,Its 2 am rn and i’m freaking out that i’m forg...,anxiety,anxiety,True
4,Son has weird bump on shoulder I'm waiting for...,anxiety,depression,False
...,...,...,...,...
2556,I have tried every single day this week I have...,suicidal,depression,False
2557,I am sorry to post this to other people dealin...,suicidal,normal,False
2558,"someone help me, my dad died yesterday, my mom...",suicidal,depression,False
2559,I am ready to go. I do not want to be here any...,suicidal,anxiety,False


In [ ]:
df = df[["statement", "status"]]
df

,statement,status
0,Driving Anxiety I tend to get very anxious whi...,anxiety
1,Department at Work Is Passing Around Stomach B...,anxiety
2,Exam on Saturday. Saturday's Starting Presenta...,anxiety
3,Its 2 am rn and i’m freaking out that i’m forg...,anxiety
4,Son has weird bump on shoulder I'm waiting for...,anxiety
...,...,...
2556,I have tried every single day this week I have...,suicidal
2557,I am sorry to post this to other people dealin...,suicidal
2558,"someone help me, my dad died yesterday, my mom...",suicidal
2559,I am ready to go. I do not want to be here any...,suicidal


In [ ]:
df["status"].value_counts()

,count
status,
anxiety,426
stress,410
personality disorder,389
suicidal,357
bipolar,335
depression,328
normal,316


In [ ]:
df["word_count"] = df.statement.apply(lambda x: len(x.split(" ")))
df = df[["statement", "status", "word_count"]]
df = df[df.word_count < 1000]

# Create a class dictionary
class_names = [
    "normal",
    "depression",
    "suicidal",
    "anxiety",
    "bipolar",
    "stress",
    "personality disorder",
]

# Define maximum samples per class
max_samples = 100

# Ensure equal distribution between all classes
df_balanced = df.groupby("status").apply(
    lambda x: x.sample(
        n=max_samples,
        replace=True if len(x) < max_samples else False
    )
)

# Reset index
df_balanced = df_balanced.reset_index(drop=True)

# Result
print(df_balanced["status"].value_counts())
df_final = df_balanced[['statement','status']]
df_final

status
anxiety                 100
bipolar                 100
depression              100
normal                  100
personality disorder    100
stress                  100
suicidal                100
Name: count, dtype: int64


<ipython-input-6-11f18c49082e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["word_count"] = df.statement.apply(lambda x: len(x.split(" ")))
<ipython-input-6-11f18c49082e>:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df.groupby("status").apply(


,statement,status
0,Health anxiety is fucking expensive. HA is fuc...,anxiety
1,Wall of Eye Floaters Help So I have floaters. ...,anxiety
2,Is there anyone here who always feels restless...,anxiety
3,Do I have a personality? I don’t feel like I h...,anxiety
4,"people seem calm, happy like there's no proble...",anxiety
...,...,...
695,I need to go soon but I am a pussy of course. ...,suicidal
696,I do not condemn anyones decision to take thei...,suicidal
697,I really want to die. Everything is pointless....,suicidal
698,All this trauma I cannot take it. I cannot han...,suicidal


In [ ]:
df_final.to_csv("mental-health-sentiment-cleaned-data.csv",index=False)

In [ ]:
df_final = df_balanced[['statement','status']]
df_final

,statement,status
0,Anxiety and dry mouth My boyfriend suffers fro...,Anxiety
1,2 Week Rule Hi everyone!\n\nI wanted to share ...,Anxiety
2,Is this a side effect of Lexapro I began takin...,Anxiety
3,How to make friends? \nI have severe social an...,Anxiety
4,I also feel like a person who is not good at b...,Anxiety
...,...,...
695,I am trying to find something that would make ...,Suicidal
696,Hello I am a 16 year old out of Germany that l...,Suicidal
697,I want to kill myself but if i go to heaven(I ...,Suicidal
698,"I am just a stupid pedo, I keep having sexual ...",Suicidal


In [ ]:
df_final['status'].value_counts()

,count
status,
Anxiety,100
Bipolar,100
Depression,100
Normal,100
Personality disorder,100
Stress,100
Suicidal,100


In [ ]:
class_names = [
    "normal",
    "depression",
    "suicidal",
    "anxiety",
    "bipolar",
    "stress",
    "personality disorder",
]
def create_prompt(statement: str, class_names: list, ctageory:str):
    # Highly structured and precise prompt
    prompt = (
        f"Classify the following text into exactly one mental health category that you need to choose from this list: {', '.join(class_names)} .\n\n"
        f"Text: {statement}\n\n"
        f"Instructions:\n"
        f"1. You must select exactly ONE category from the following list: {', '.join(class_names)}.\n"
        f"2. Do not create new categories. If the exact match is unclear, choose the closest and most appropriate category from this list : {', '.join(class_names)} .\n"
        f"3. Respond with ONLY the category name (no explanations, comments, or additional text).\n"
        f"4. The category must match one of the provided names exactly as written (case-insensitive) like the list of categoris that you need to choose from list: {', '.join(class_names)} .\n\n"
        f"5. your response schould be one word or max two words and schould be one from these categories {', '.join(class_names)} "
        f"Category: {category}"
    )
    return prompt
for i in range(len(df_final)):
  category = df_final['status'][i]
  df_final['statement'][i] = create_prompt(df_final['statement'][i], class_names, category)

df_final

<ipython-input-7-1c6e43d108dc>:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_final['statement'][i] = create_prompt(df_final['statement'][i], class_names, category)
<ipython-input-7-1c6e43d108dc>:26: SettingWithCopyWarning: 
A value is

,statement,status
0,Classify the following text into exactly one m...,anxiety
1,Classify the following text into exactly one m...,anxiety
2,Classify the following text into exactly one m...,anxiety
3,Classify the following text into exactly one m...,anxiety
4,Classify the following text into exactly one m...,anxiety
...,...,...
695,Classify the following text into exactly one m...,suicidal
696,Classify the following text into exactly one m...,suicidal
697,Classify the following text into exactly one m...,suicidal
698,Classify the following text into exactly one m...,suicidal


In [ ]:
df_final.to_csv("prompt_data.csv",index=False)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    get_cosine_with_hard_restarts_schedule_with_warmup
)
from tqdm import tqdm
import pandas as pd
import wandb
import random
import numpy as np
from sklearn.metrics import f1_score, classification_report
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
class_names = [
    "normal",
    "depression",
    "suicidal",
    "anxiety",
    "bipolar",
    "stress",
    "personality disorder"
]
def calculate_metrics(predictions, targets):
    """Calculate accuracy and F1 scores (micro, macro, weighted)"""
    if len(predictions) == 0:
        return {
            'accuracy': 0,
            'f1_micro': 0,
            'f1_macro': 0,
            'f1_weighted': 0,
            'class_metrics': {c: {'precision': 0, 'recall': 0, 'f1': 0} for c in class_names}
        }

    correct = sum(1 for p, t in zip(predictions, targets) if p == t)
    accuracy = correct / len(predictions)

    f1_micro = f1_score(targets, predictions, average='micro')
    f1_macro = f1_score(targets, predictions, average='macro')
    f1_weighted = f1_score(targets, predictions, average='weighted')

    report = classification_report(targets, predictions, output_dict=True)
    class_metrics = {
        class_name: {
            'precision': report.get(class_name, {}).get('precision', 0),
            'recall': report.get(class_name, {}).get('recall', 0),
            'f1': report.get(class_name, {}).get('f1-score', 0)
        }
        for class_name in class_names
    }

    return {
        'accuracy': accuracy,
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
        'class_metrics': class_metrics
    }
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

class MentalHealthDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        input_text = text.split("Category:")[0] + "Category:"
        target_text = text.split("Category:")[-1].strip()

        encoding = self.tokenizer(
            input_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        target_encoding = self.tokenizer(
            target_text,
            max_length=20,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': target_encoding['input_ids'].flatten(),
            'target_text': target_text
        }

    def __len__(self):
        return len(self.texts)

def train_epoch(model,tokenizer, dataloader, criterion, optimizer, scheduler, device, epoch):
    model.train()
    total_loss = 0
    progress_bar = tqdm(dataloader, desc=f'Training Epoch {epoch + 1}')

    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)


        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        shift_logits = outputs.logits[:, -labels.size(1):, :].contiguous()

        predicted_tokens = torch.argmax(shift_logits, dim=-1)

        loss = criterion(shift_logits.view(-1, shift_logits.size(-1)), labels.view(-1))

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        progress_bar.set_postfix({'loss': loss.item()})

    avg_loss = total_loss / len(dataloader)
    print(f"\nEpoch {epoch + 1} - Average Training Loss: {avg_loss:.4f}")
    return avg_loss

def evaluate(model, tokenizer, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(dataloader, desc='Evaluating')):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            target_texts = batch['target_text']

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            shift_logits = outputs.logits[:, -labels.size(1):, :].contiguous()
            predicted_tokens = torch.argmax(shift_logits, dim=-1)
            loss = criterion(shift_logits.view(-1, shift_logits.size(-1)), labels.view(-1))
            total_loss += loss.item()

            for i in range(len(predicted_tokens)):
                pred_text = tokenizer.decode(predicted_tokens[i], skip_special_tokens=True).strip().lower()
                target = target_texts[i].strip().lower()

                normalized_pred = None
                for class_name in class_names:
                    if class_name in pred_text:
                        normalized_pred = class_name
                        break

                if normalized_pred is not None:
                    all_predictions.append(normalized_pred)
                    all_targets.append(target)

    if len(all_predictions) > 0:
        print(f"\nTotal predictions made: {len(all_predictions)}")
        print("Sample predictions:")
        for p, t in list(zip(all_predictions, all_targets))[:5]:
            print(f"Pred: {p} | Target: {t}")

        metrics = calculate_metrics(all_predictions, all_targets)
    else:
        print("\nWarning: No valid predictions were made!")
        metrics = {
            'accuracy': 0,
            'f1_micro': 0,
            'f1_macro': 0,
            'f1_weighted': 0,
            'class_metrics': {c: {'precision': 0, 'recall': 0, 'f1': 0} for c in class_names}
        }

    metrics['loss'] = total_loss / len(dataloader)
    metrics['predictions'] = all_predictions
    metrics['targets'] = all_targets

    # Print evaluation metrics
    print("\nEvaluation Metrics:")
    print(f"Validation Loss: {metrics['loss']:.4f}")
    print(f"Validation Accuracy: {metrics['accuracy']:.4f}")
    print(f"F1 Score (micro): {metrics['f1_micro']:.4f}")
    print(f"F1 Score (macro): {metrics['f1_macro']:.4f}")
    print(f"F1 Score (weighted): {metrics['f1_weighted']:.4f}")

    return metrics

def train_and_evaluate(model, tokenizer, train_dataloader, val_dataloader, criterion, optimizer, scheduler, device, config):
    best_train_loss = float('inf')
    best_val_loss = float('inf')
    best_f1 = 0
    patience = 0
    max_patience = 3

    for epoch in range(config['epochs']):
        print(f"\n{'='*50}")
        print(f"Epoch {epoch + 1}/{config['epochs']}")
        print(f"{'='*50}")

        train_loss = train_epoch(model, tokenizer, train_dataloader, criterion, optimizer, scheduler, device, epoch)

        eval_metrics = evaluate(model, tokenizer, val_dataloader, criterion, device)
        val_loss = eval_metrics['loss']
        current_f1 = eval_metrics['f1_weighted']

        wandb.log({
            'train_loss': train_loss,
            'val_loss': val_loss,
            'val_f1_weighted': current_f1,
            'epoch': epoch + 1
        })

        if train_loss < best_train_loss and val_loss < best_val_loss and current_f1 >= best_f1:
            print("\nNew best model found!")
            print(f"Train Loss: {train_loss:.4f} (prev: {best_train_loss:.4f})")
            print(f"Val Loss: {val_loss:.4f} (prev: {best_val_loss:.4f})")
            print(f"F1 Score: {current_f1:.4f} (prev: {best_f1:.4f})")

            model.save_pretrained('best_model')
            tokenizer.save_pretrained('best_model')

            best_train_loss = train_loss
            best_val_loss = val_loss
            best_f1 = current_f1
            patience = 0
        else:
            patience += 1
            print(f"\nNo improvement. Patience: {patience}/{max_patience}")

            if patience >= max_patience:
                print(f"\nEarly stopping triggered after {epoch + 1} epochs")
                break

    return {
        'best_train_loss': best_train_loss,
        'best_val_loss': best_val_loss,
        'best_f1': best_f1
    }

def main():
    config = {
        'model_name': "Qwen/Qwen2.5-0.5B-Instruct",
        'max_length': 786,
        'batch_size': 2,
        'learning_rate': 3e-5,
        'epochs': 15,
        'warmup_ratio': 0.1,
        'weight_decay': 0.01,
        'seed': 42,
        'gradient_accumulation_steps': 8,
        'max_grad_norm': 1.0,
        'metric_for_best_model': 'f1_weighted',
        # LoRA specific configs
        'lora_r': 8,
        'lora_alpha': 32,
        'lora_dropout': 0.1
    }

    wandb.init(project="mental-health-classification-lora", config=config)
    set_seed(config['seed'])

    df = pd.read_csv('/content/drive/MyDrive/prompt_data.csv')

    train_df = df.sample(frac=0.8, random_state=config['seed'])
    val_df = df.drop(train_df.index)

    tokenizer = AutoTokenizer.from_pretrained(config['model_name'], trust_remote_code=True)
    tokenizer.padding_side = 'left'
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        config['model_name'],
        trust_remote_code=True,
        pad_token_id=tokenizer.pad_token_id,
        device_map='auto'
    )

    model = prepare_model_for_kbit_training(model)

    lora_config = LoraConfig(
        r=config['lora_r'],
        lora_alpha=config['lora_alpha'],
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=config['lora_dropout'],
        bias="none",
        task_type=TaskType.CAUSAL_LM
    )

    # Create PEFT model
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    train_dataset = MentalHealthDataset(train_df['statement'].values, tokenizer, config['max_length'])
    val_dataset = MentalHealthDataset(val_df['statement'].values, tokenizer, config['max_length'])

    train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    scaler = torch.cuda.amp.GradScaler()

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=config['learning_rate'],
        weight_decay=config['weight_decay']
    )

    num_training_steps = len(train_dataloader) * config['epochs']
    num_warmup_steps = int(num_training_steps * config['warmup_ratio'])
    scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps
    )


    criterion = nn.CrossEntropyLoss()
    best_f1 = 0
    patience = 0
    max_patience = 3

    print("\nStarting Training...")
    results = train_and_evaluate(
        model=model,
        tokenizer=tokenizer,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        config=config
    )

    print("\nTraining completed!")
    print(f"Best metrics achieved:")
    print(f"Train Loss: {results['best_train_loss']:.4f}")
    print(f"Validation Loss: {results['best_val_loss']:.4f}")
    print(f"F1 Score: {results['best_f1']:.4f}")

    wandb.finish()

if __name__ == "__main__":
    main()

<ipython-input-14-d115b5ffea35>:328: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


trainable params: 1,081,344 || all params: 495,114,112 || trainable%: 0.2184

Starting Training...

Epoch 1/15


Training Epoch 1: 100%|██████████| 280/280 [05:23<00:00,  1.16s/it, loss=0.279]



Epoch 1 - Average Training Loss: 5.2016


Evaluating: 100%|██████████| 70/70 [00:37<00:00,  1.85it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

...

Training Epoch 7: 100%|██████████| 280/280 [05:26<00:00,  1.17s/it, loss=0.0526]



Epoch 7 - Average Training Loss: 0.1093


Evaluating: 100%|██████████| 70/70 [00:37<00:00,  1.84it/s]



Total predictions made: 129
Sample predictions:
Pred: anxiety | Target: anxiety
Pred: anxiety | Target: anxiety
Pred: anxiety | Target: anxiety
Pred: anxiety | Target: anxiety
Pred: anxiety | Target: anxiety

Evaluation Metrics:
Validation Loss: 0.1697
Validation Accuracy: 0.6977
F1 Score (micro): 0.6977
F1 Score (macro): 0.6906
F1 Score (weighted): 0.6950

No improvement. Patience: 1/3

Epoch 8/15


Training Epoch 8:  55%|█████▌    | 154/280 [02:59<02:27,  1.17s/it, loss=0.00816]

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/prompt_data.csv')

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "/content/best_model",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("/content/best_model")

input_text = str(df['statement'][405]).split("Category:")[0] + "Category:"
target_text = str(df['statement'][405]).split("Category:")[-1].strip()

input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)
label_ids = tokenizer.encode(target_text, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model(input_ids=input_ids)

logits = outputs.logits[:, -label_ids.size(1):, :].contiguous()
predicted_tokens = torch.argmax(logits, dim=-1)
pred_text = tokenizer.decode(predicted_tokens[0], skip_special_tokens=True).strip().lower()

print("Generated Output:")
print(pred_text)
print('*' * 50)
print("True label:")
print(target_text)


Generated Output:
personality disorder
**************************************************
True label:
personality disorder


In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
torch.cuda.empty_cache()
import gc
gc.collect()
torch.cuda.empty_cache()
!nvidia-smi

Tue Jan  7 23:23:18 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install huggingface_hub


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `AmenTest` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `AmenTest`

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import HfApi

# Load your model and tokenizer
model_name = "/content/best_model"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Push to Hugging Face Hub
repo_name = "Amen002365/Qwen2.5-0.5B-finetuned-on-mental-health-dataset"  # Change to your desired repo name
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

print(f"Model and tokenizer saved to: https://huggingface.co/{repo_name}")


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/4.35M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Model and tokenizer saved to: https://huggingface.co/Amen002365/Qwen2.5-0.5B-finetuned-on-mental-health-dataset


In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "Amen002365/Qwen2.5-0.5B-finetuned-on-mental-health-dataset",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Amen002365/Qwen2.5-0.5B-finetuned-on-mental-health-dataset")


input_text = str(df['statement'][405]).split("Category:")[0] + "Category:"
target_text = str(df['statement'][405]).split("Category:")[-1].strip()

input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)
label_ids = tokenizer.encode(target_text, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model(input_ids=input_ids)


logits = outputs.logits[:, -label_ids.size(1):, :].contiguous()
predicted_tokens = torch.argmax(logits, dim=-1)
pred_text = tokenizer.decode(predicted_tokens[0], skip_special_tokens=True).strip().lower()

print("Generated Output:")
print(pred_text)
print('*' * 50)
print("True label:")
print(target_text)


config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/4.35M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.53k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

Generated Output:
personality disorder
**************************************************
True label:
personality disorder
